In [ ]:
from __future__ import division
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from caspyr.utils import H5Reader
from caspyr.plotting import volshow
from argopt import DictAttrWrap
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
def imshow(im, cmap="Greys", origin="lower", title=None):
    ax = plt.imshow(im, cmap=cmap, origin=origin)
    plt.axis("off")
    if title:
        plt.title(title)

In [ ]:
dat = H5Reader("brainweb.raws/subject_04_sigma1_noise0.75_mMR.mat", prefix="MultiMaps_Ref/")
dat.PET = H5Reader("brainweb.raws/subject_04_sigma1_noise1_mMR.mat", prefix="MultiMaps_Ref/").PET[:]
d = dict()
for i in dat.keys():
    d[i] = getattr(dat, i)[:]
del dat
d = DictAttrWrap(d)

In [ ]:
ROI = slice(0, None), slice(100, -100), slice(100, -100)
volshow([(i, getattr(d, i)[ROI]) for i in d.d.keys()]);

In [ ]:
nReals = 5
PAD = 4
ROI = (19, 108), (128, 100+115), (120, 100+124)
ROI = [slice(i - PAD, j + PAD) for i, j in ROI]

In [ ]:
def load(nPats, nReals=5, counts=None, ROI=None, **kwargs):
    """
    Intended for loading training data, e.g.:
    >>> x, y = load(10, 1, counts=[30, 300])

    @param counts  : [low, high] in millions [default: [3, 300]]
    @param ROI  : ZXY (even though return is ZYX)
    @return ndarray, shape (len(counts), nPats*nReals, D, H, W, Ch)
      Ch is 3=<T1, PSF, PET> if `counts > 0`, 1=<TRUTH> otherwise.
    """
    from caspyr.utils import Globber, H5Reader
    from os import path
    import numpy as np
    import logging
    from tqdm.auto import tqdm
    glob = Globber.glob
    log = logging.getLogger(__name__)

    DAT_ROOT = "/data/cc16/apirl/o"
    #"brainweb_PETpsf_1_subject_{}-S_1-NP_1-NT1_0.75-C_{:.3g}_t3_{:03d}.mat"
    subjects = sorted({i[47:49] for i in glob(path.join(
        DAT_ROOT, "reconAPIRL_brainweb_subject_*-S_1-NP_1-NT1_0.75-C_*_t3.mat"))})
    log.debug("subj:" + ', '.join(subjects))

    counts = counts or (3.01, 301)
    ROI = ROI or (slice(0, None),) * 3
    assert len(ROI) == 3
    ROI = tuple(ROI)

    res = [] # [counts, nReals + nReals_psf + T1 + PET, z,y,x]
    for c in tqdm(counts, desc="counts"):
        vols = []
        for subj in tqdm(subjects[:nPats], desc="subject", leave=False):
            d = path.join(DAT_ROOT,
                "reconAPIRL_brainweb_subject_{}-S_1-NP_1-NT1_0.75-C_{:.3g}_t3.mat".format(
                    subj, abs(c)*1e6))
            d = H5Reader(d, prefix="reconAPIRL")
            if c < 0:
                i = d.PET[ROI][None, :,:,:, None] * d.scale_factor[0,0]  # 1zyx1
                i = np.repeat(i, min(d.mlem.shape[3], nReals), axis=-1)  # 1zyxN
            else:
                i = np.concatenate((
                    d.mlem_psf[ROI + (slice(0, nReals),)][None],
                    d.mlem[ROI + (slice(0, nReals),)][None],
                ))  # 2zyxN
                i = np.concatenate((np.repeat(d.T1[ROI][None, :,:,:, None], i.shape[-1], axis=-1), i))  # 3zyxN
                #d.PET[ROI][None, :,:,:,None] * d.scale_factor[0,0]
            i = i.transpose((4, 1,3,2, 0))  # Nzxy3
            vols.append(i)
        res.append(np.concatenate(vols))
    return res

# low->high
#dat = load(10, nReals=nReals, counts=[3.01, 301], ROI=ROI) # 2, nReals*10, z, y, x, 3
#np.savez_compressed('bweb-3.01-301.npz', x=dat[0], y=dat[1])

dat = load(10, nReals=nReals, counts=[3.01, -3.01], ROI=ROI)
np.savez_compressed('bweb-3.01-T.npz', x=dat[0], y=dat[1])

#dat = load(10, nReals=nReals, counts=[30.1, -30.1], ROI=ROI)
#np.savez_compressed('bweb-30.1-T.npz', x=dat[0], y=dat[1])

In [ ]:
d.uMap = np.fromfile('output/data-LM-00-umap-AD_1.v', dtype=np.float32).reshape(127, 344, 344).transpose((0, 2, 1))

In [ ]:
fig, axs = plt.subplots(2, 2, sharex=True, sharey=True)

In [ ]:
plt.sca(axs.flat[0]); imshow(d.PET[64, 100:-100, 100:-100], title="PET")
plt.sca(axs.flat[1]); imshow(d.T1[64, 100:-100, 100:-100], title="T1")
plt.sca(axs.flat[2]); imshow(d.PET_psf[64, 100:-100, 100:-100], title="PET_psf")
plt.sca(axs.flat[3]); imshow(d.uMap[64, 100:-100, 100:-100], title="uMap")